In [248]:
import pandas as pd

# Options

In [249]:
all_options = pd.read_csv('../data/SPYOPT_filtered.csv',parse_dates=['ts_event'])

In [250]:
all_options = all_options.drop(columns=['rtype', 'publisher_id','instrument_id'])

In [251]:
def split_symbol(symbol):
    exp_my,type_price = symbol.split(' ')
    option_type = type_price[0]
    strike = type_price[1:]
    month = exp_my[2]
    year = exp_my[3]
    return month, year, option_type, strike
    

In [252]:
symbol_info = pd.DataFrame(all_options['symbol'].apply(split_symbol).to_list(),columns=['month','year','option_type','strike'])

In [253]:
month_map = {'F': '01', 'G': '02', 'H': '03', 'J': '04', 'K': '05', 'M': '06',
             'N': '07', 'Q': '08', 'U': '09', 'V': '10', 'X': '11', 'Z': '12'}


In [254]:
symbol_info['month'] = symbol_info['month'].map(month_map)

In [255]:
from datetime import datetime, date,timedelta

def get_expiration_date(month,year, todays_date):
    year +=10

    if date(year+2000, month, 28) < todays_date.date():
        year += 10

    year = int(year) + 2000
    
    first_of_month = date(year, month, 1)
    days_to_first_friday = (4 - first_of_month.weekday()) % 7
    first_friday = first_of_month + timedelta(days=days_to_first_friday)
    third_friday_date = first_friday + timedelta(weeks=2)
    return third_friday_date







In [256]:
all_options = pd.concat([all_options, symbol_info], axis=1)

In [257]:
all_options.dtypes

ts_event       datetime64[ns]
open                  float64
high                  float64
low                   float64
close                 float64
volume                  int64
symbol                 object
month                  object
year                   object
option_type            object
strike                 object
dtype: object

In [258]:
expiration_date = all_options.apply(lambda row: get_expiration_date(int(row['month']), int(row['year']), pd.to_datetime(row['ts_event'])), axis=1)

In [259]:
all_options['expiration_date'] = expiration_date

all_options = all_options.drop(columns=['symbol', 'month', 'year'])

In [260]:
all_options.to_csv('../data/SPYOPT_processed.csv', index=False)

# Futures

In [279]:
futures = pd.read_csv('../data/SPYFUT.csv')
futures = futures.drop(columns=['rtype', 'publisher_id','instrument_id'])

In [280]:
futures_filtered = futures[~futures['symbol'].str.contains('-')]
futures_filtered = futures_filtered.dropna()

In [281]:
def split_futures_symbol(symbol):
    month = symbol[2]
    year = symbol[3]
    return month, year

In [282]:
dates = pd.DataFrame(futures_filtered['symbol'].apply(split_futures_symbol).to_list(), columns=['month', 'year'])
dates['month'] = dates['month'].map(month_map)

In [283]:
futures_filtered = pd.concat([
    futures_filtered.reset_index(drop=True),
    dates.reset_index(drop=True)
], axis=1)

In [285]:
futures_filtered ['expiration_date'] = futures_filtered.apply(
    lambda row: get_expiration_date(int(row['month']), int(row['year']), pd.to_datetime(row['ts_event'])), axis=1)

In [287]:
futures_filtered = futures_filtered.drop(columns=['symbol', 'month', 'year'])

In [289]:
futures_filtered.to_csv('../data/SPYFUT_processed.csv', index=False)